In [10]:
import sys
import os
sys.path.insert(0, r'C:\Users\hadrien.lipka\src')
from Football.python_scripts.support_functions import communicate_AI,create_prompt,merge_days,merge_seasons,create_excel_database,AI_answer_to_dict,AI_answer_to_df
import pandas as pd
from itertools import product

In [2]:
# create cases matrix
leagues=['Ligue 1']
countries=['France']
seasons=[[2018,2019]]
days=[i for i in range(1,39)] # 38 rounds
feed = list(product(seasons, countries, leagues, days))
MATRIX_CASES = pd.DataFrame(feed, columns=['season','country','league','day'])
MATRIX_CASES.head()

,season,country,league,day
0,"[2018, 2019]",France,Ligue 1,1
1,"[2018, 2019]",France,Ligue 1,2
2,"[2018, 2019]",France,Ligue 1,3
3,"[2018, 2019]",France,Ligue 1,4
4,"[2018, 2019]",France,Ligue 1,5


In [3]:
%%time
all_days_2018_2019_Ligue_1_France = []
for i, row in MATRIX_CASES.iterrows():
    if row.season==[2018,2019] and row.league=='Ligue 1' and row.country=='France':
        prompt = create_prompt(league=row.league, country=row.country, season=row.season, day=row.day)
        picture_name = f'{row.season[0]}_{row.season[-1]}_{row.league.replace(' ', '_')}_{row.country}_{row.day}'
        answer = communicate_AI(prompt=prompt, picture_name=picture_name)
        #df = AI_answer_to_df(answer=answer) # the results df for unique day
        df = pd.DataFrame(AI_answer_to_dict(answer=answer))
        assert df.shape[0]==10, f"Day {row.day} doesn't have 10 games"
        all_days_2018_2019_Ligue_1_France.append(df)
print(len(all_days_2018_2019_Ligue_1_France))

38
CPU times: total: 984 ms
Wall time: 4min 37s


In [4]:
full_2018_2019_Ligue_1_France_df = merge_days(all_days_2018_2019_Ligue_1_France)
full_2018_2019_Ligue_1_France_df

,season,country,league,day,home_team,away_team,home_score,away_score
0,2018-2019,France,Ligue 1,1,PSG,Caen,3,0
1,2018-2019,France,Ligue 1,1,Bordeaux,Strasbourg,0,2
2,2018-2019,France,Ligue 1,1,Lyon,Amiens,2,0
3,2018-2019,France,Ligue 1,1,Angers,Nimes,3,4
4,2018-2019,France,Ligue 1,1,Lille,Rennes,3,1
...,...,...,...,...,...,...,...,...
375,2018-2019,France,Ligue 1,38,Marseille,Montpellier,1,0
376,2018-2019,France,Ligue 1,38,Nantes,Strasbourg,0,1
377,2018-2019,France,Ligue 1,38,Nice,Monaco,2,0
378,2018-2019,France,Ligue 1,38,Nimes,Lyon,2,3


In [5]:
full_2018_2019_Ligue_1_France_df.home_team.unique()

array(['PSG', 'Bordeaux', 'Lyon', 'Angers', 'Lille', 'Montpellier',
       'Nice', 'St Etienne', 'Nantes', 'Marseille', 'Reims', 'Guingamp',
       'Rennes', 'Monaco', 'Dijon', 'Strasbourg', 'Amiens', 'Toulouse',
       'Nimes', 'Caen', 'Nîmes'], dtype=object)

In [6]:
full_2018_2019_Ligue_1_France_df['home_team'] = full_2018_2019_Ligue_1_France_df['home_team'].replace({'Nîmes': 'Nimes',
                                                                                                       'Saint-Étienne': 'St Etienne'})
full_2018_2019_Ligue_1_France_df['away_team'] = full_2018_2019_Ligue_1_France_df['away_team'].replace({'Nîmes': 'Nimes',
                                                                                                       'Saint-Étienne': 'St Etienne'})
full_2018_2019_Ligue_1_France_df

,season,country,league,day,home_team,away_team,home_score,away_score
0,2018-2019,France,Ligue 1,1,PSG,Caen,3,0
1,2018-2019,France,Ligue 1,1,Bordeaux,Strasbourg,0,2
2,2018-2019,France,Ligue 1,1,Lyon,Amiens,2,0
3,2018-2019,France,Ligue 1,1,Angers,Nimes,3,4
4,2018-2019,France,Ligue 1,1,Lille,Rennes,3,1
...,...,...,...,...,...,...,...,...
375,2018-2019,France,Ligue 1,38,Marseille,Montpellier,1,0
376,2018-2019,France,Ligue 1,38,Nantes,Strasbourg,0,1
377,2018-2019,France,Ligue 1,38,Nice,Monaco,2,0
378,2018-2019,France,Ligue 1,38,Nimes,Lyon,2,3


In [7]:
full_2018_2019_Ligue_1_France_df.away_team.unique()

array(['Caen', 'Strasbourg', 'Amiens', 'Nimes', 'Rennes', 'Dijon',
       'Reims', 'Guingamp', 'Monaco', 'Toulouse', 'Lyon', 'PSG', 'Angers',
       'Lille', 'Nantes', 'St Etienne', 'Montpellier', 'Bordeaux',
       'Marseille', 'Nice'], dtype=object)

In [8]:
print(len(full_2018_2019_Ligue_1_France_df.home_team.unique().tolist()), len(full_2018_2019_Ligue_1_France_df.away_team.unique().tolist()))

20 20


In [9]:
INSERT_DB = False
if INSERT_DB:
    create_excel_database(full_2018_2019_Ligue_1_France_df)

⚠️ The file '..\database.xlsx' already exists. Overwrite? (y/n):  y


✅ Data exported to '..\database.xlsx' successfully.
